In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import torch
from torch import nn
from torch import optim
import torch.optim as optim
import torch.nn.functional as F
from torchvision import transforms, models

In [3]:
from torch.autograd import Variable
import torch.utils.data as data
import pandas as pd
import os
from PIL import Image
from torch.utils.data import DataLoader, TensorDataset


In [4]:
# Path for (unlabeled) test data
test_path = '/content/drive/MyDrive/패턴-TermProject/TestImages/'

In [5]:
# Prepare test data set

data =[]
label =[]
resize = transforms.Compose([transforms.CenterCrop(450)]) #crop to 400

files = os.listdir(test_path)
files.sort(); # so that resutls can be printed in the order of 'test01', 'test02', ... 'test80'

print(files)

for f in files:
  img = Image.open(test_path+'/'+f,'r')
  img = resize(img)
  r,g,b = img.split()
  one_img = np.asarray(np.float32(r)/255.0)
  img = np.asarray([one_img])
  data.append(img)
  label.append(1) # dummy numbers (not to be used anyway)

data = np.array(data, dtype='float32')
label = np.array(label, dtype='int64')

test_X = torch.from_numpy(data)
test_Y = torch.from_numpy(label)

test_X = test_X.type(torch.cuda.FloatTensor)
test_Y = test_Y.type(torch.cuda.LongTensor)

test_dataset = TensorDataset(test_X, test_Y)
#test_dataset = TensorDataset(test_X)
test_loader = DataLoader(test_dataset, batch_size = 20, shuffle=False) # set batch size to 80 to print all the predictions in one vector



['test01.jpg', 'test02.jpg', 'test03.jpg', 'test04.jpg', 'test05.jpg', 'test06.jpg', 'test07.jpg', 'test08.jpg', 'test09.jpg', 'test10.jpg', 'test11.jpg', 'test12.jpg', 'test13.jpg', 'test14.jpg', 'test15.jpg', 'test16.jpg', 'test17.jpg', 'test18.jpg', 'test19.jpg', 'test20.jpg', 'test21.jpg', 'test22.jpg', 'test23.jpg', 'test24.jpg', 'test25.jpg', 'test26.jpg', 'test27.jpg', 'test28.jpg', 'test29.jpg', 'test30.jpg', 'test31.jpg', 'test32.jpg', 'test33.jpg', 'test34.jpg', 'test35.jpg', 'test36.jpg', 'test37.jpg', 'test38.jpg', 'test39.jpg', 'test40.jpg', 'test41.jpg', 'test42.jpg', 'test43.jpg', 'test44.jpg', 'test45.jpg', 'test46.jpg', 'test47.jpg', 'test48.jpg', 'test49.jpg', 'test50.jpg', 'test51.jpg', 'test52.jpg', 'test53.jpg', 'test54.jpg', 'test55.jpg', 'test56.jpg', 'test57.jpg', 'test58.jpg', 'test59.jpg', 'test60.jpg', 'test61.jpg', 'test62.jpg', 'test63.jpg', 'test64.jpg', 'test65.jpg', 'test66.jpg', 'test67.jpg', 'test68.jpg', 'test69.jpg', 'test70.jpg', 'test71.jpg', 'test

In [6]:
class VGG(nn.Module):
    def __init__(self, features, num_classes=2, init_weights=True):
        super(VGG, self).__init__()
        
        self.features = features #convolution
        
        self.avgpool = nn.AdaptiveAvgPool2d((7, 7))
        
        self.classifier = nn.Sequential(
            nn.Linear(12544, 4096),
            nn.ReLU(True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(True),
            nn.Dropout(),
            nn.Linear(4096, num_classes),
        )#FC layer
        
        if init_weights:
            self._initialize_weights()

    def forward(self, x):
        x = self.features(x) #Convolution 
        x = self.avgpool(x) # avgpool
        x = x.view(x.size(0), -1) #
        x = self.classifier(x) #FC layer
        return x

    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.normal_(m.weight, 0, 0.01)
                nn.init.constant_(m.bias, 0)

def make_layers(cfg, batch_norm=False):
    layers = []
    in_channels = 1
    
    for v in cfg:
        if v == 'M':
            layers += [nn.MaxPool2d(kernel_size=2, stride=2)]
        else:
            conv2d = nn.Conv2d(in_channels, v, kernel_size=3, padding=1)
            if batch_norm:
                layers += [conv2d, nn.BatchNorm2d(v), nn.ReLU(inplace=True)]
            else:
                layers += [conv2d, nn.ReLU(inplace=True)]
            in_channels = v
                     
    return nn.Sequential(*layers)
    
cfg = { 'custom' : [64,'M',128,'M',256,256,'M']}

model = VGG(make_layers(cfg['custom']), num_classes=2, init_weights=True)

In [7]:
model = torch.load('/content/drive/MyDrive/패턴-TermProject/model_vgg_epoch37.pth')

In [13]:
model.eval()
criterion = nn.CrossEntropyLoss()

for inputs, labels in test_loader:
    inputs, labels = Variable(inputs), Variable(labels)
    output = model(inputs)
    idx, pred = output.max(1)
    pred = (pred.detach().cpu().numpy())
    print('Test results : {}'.format(pred))
    # print(pred)

Test results : [0 0 1 1 0 1 0 1 0 0 0 1 0 0 0 1 0 0 0 0]
Test results : [0 1 0 0 1 1 0 0 1 1 0 1 0 1 1 1 1 0 1 1]
Test results : [0 0 0 0 1 0 1 1 1 0 1 0 0 0 0 0 0 0 1 1]
Test results : [0 1 1 0 0 1 0 0 1 0 0 1 0 0 0 0 0 1 0 1]
Test results : [0 0 0 1 0 1 0 1 0 1]


In [14]:
all_result = [0,0,1,1,0,1,0,1,0,0,0,1,0,0,0,1,0,0,0,0]
all_result += [0,1,0,0,1,1,0,0,1,1,0,1,0,1,1,1,1,0,1,1]
all_result += [0,0,0,0,1,0,1,1,1,0,1,0,0,0,0,0,0,0,1,1]
all_result += [0,1,1,0,0,1,0,0,1,0,0,1,0,0,0,0,0,1,0,1]
all_result += [0,0,0,1,0,1,0,1,0,1]
print(all_result)
print(len(all_result))

[0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1]
90


In [15]:
name = []
for i in range(90):
  name.append(files[i].split('.')[0])
name

['test01',
 'test02',
 'test03',
 'test04',
 'test05',
 'test06',
 'test07',
 'test08',
 'test09',
 'test10',
 'test11',
 'test12',
 'test13',
 'test14',
 'test15',
 'test16',
 'test17',
 'test18',
 'test19',
 'test20',
 'test21',
 'test22',
 'test23',
 'test24',
 'test25',
 'test26',
 'test27',
 'test28',
 'test29',
 'test30',
 'test31',
 'test32',
 'test33',
 'test34',
 'test35',
 'test36',
 'test37',
 'test38',
 'test39',
 'test40',
 'test41',
 'test42',
 'test43',
 'test44',
 'test45',
 'test46',
 'test47',
 'test48',
 'test49',
 'test50',
 'test51',
 'test52',
 'test53',
 'test54',
 'test55',
 'test56',
 'test57',
 'test58',
 'test59',
 'test60',
 'test61',
 'test62',
 'test63',
 'test64',
 'test65',
 'test66',
 'test67',
 'test68',
 'test69',
 'test70',
 'test71',
 'test72',
 'test73',
 'test74',
 'test75',
 'test76',
 'test77',
 'test78',
 'test79',
 'test80',
 'test81',
 'test82',
 'test83',
 'test84',
 'test85',
 'test86',
 'test87',
 'test88',
 'test89',
 'test90']

In [20]:
cd /content/drive/MyDrive/패턴-TermProject

/content/drive/MyDrive/패턴-TermProject


In [22]:
# csv 파일로 저장

import csv

with open('result.csv', 'w', newline='') as f: 
  writer = csv.writer(f) 
  for i in range(len(name)):
    temp = [name[i], all_result[i]]
    writer.writerow(temp) 
